In [ ]:
# Get input query

query = str(input())


query = text_formatting(query)
query = query.split(" ")



In [ ]:
with open("vocabulary.??",encoding = "utf-8")    as f:
    
vocabulary = f.split("\n")                         # Splitting each word in a new Row, how?

term_id_list = list()                                       # Creating a list for words-indexes
doc_list = list()                                           # Creating a list of matching documents


for word in query:                                 # Searching for each word in the query into the vocabulary  
    
    
    # If the word is in the vocabulary
    if w in vocabulary:
        
        # Get the vocabulary index for the word
        term_id = vocabulary.index(w)
        
        # Append it to the list of term_ids
        term_id_list.append(term_id)
    
    # Solo i doc con tutte le parole? altrimenti non serve
    else:
        term_id_list = list()
        break
                


# Get list of documetns containing the words (term_ids) just found
for term_id in term_id_list:
    if term_id in 'synopsis_index.json': #come si puo mettere?
        doc_list.append(dictionary[term_id])

In [ ]:
# Compute the intersection of the sets of documents corresponding to each term_id we have found

if len(doc_list) > 0:
    selected_docs = set(doc_list[0])

    for l in doc_list:
        selected_docs = selected_docs.intersection(set(l))
else:
    selected_docs = list()
    
selected_docs = list(selected_docs)

we are now going to implement a scoring system. The general principle is to compute the distance between the query and each one of the documents. This is done by creating a vector for each one of them. Each component of the vectors corresponds to a word. In the query vector, if the word is contained, the component will be $1$, otherwise it will be $0$. In the documents vectors, the components are the $TFIDF = TF \cdot IDF$ of each word in the given document.</p>

The TF (term frequency) is defined as:

TF = nw/Nw

where nw is the number of occurences of the word in document, and Nw the total number of words in the document.

while the IDF (inverse document frequency) is:

IDF = log(Nd/nd)

where Nd is the total number of documents and nd the number of documents containing the word

In [ ]:
vocabulary_file = open("vocabulary.json", "r", 
                       encoding = "utf-8")                     # Open the vocabulary file
vocabulary = vocabulary_file.read()                            # Reading 
vocabulary = vocabulary.split("\n")

  

    
# For every tsv
# Open file
    file = open(file_path  ".tsv", "r", encoding = "utf-8")#come li prendo?

    # Read entry
    entry = file.read()
    entry = entry.split("\t")

    # Get description and title
    description = entry[10]
    title = entry[0]

    # Merge in a single string variable the title and the description
    des_tit = description + " " + title
    des_tit = des_tit.split(" ")

    # Compute TFIDF
    
    # Def a counter 
    counter = Counter(des_tit)
    
    # For every word in the description and title
    for word in set(des_tit):
        
        # Compute the term frequency in the given document
        tf = (counter[word]/len(des_tit))
        
        # Get the term_id from the vocabulary
        term_id = vocabulary.index(word)
    
        # Compute the inverse document frequency
        N = 19124 #how much file?  
        n = len(dizionario[term_id]) #come la prendo questa?
        idf = math.log10(N/n)
    
        # Compute the TFIDF
        tfidf = tf*idf

        # Second inverted index
        
        dictionary2= dict()
        if (term_id not in dictionary2):
            dictionary2[term_id] = [(doc_index, tfidf)]      
        else (term_id in dictionary2):                       
            dictionary2[term_id].append((doc_index, tfidf))  

In [ ]:
# inverted index in a file

file = open("inverted_index2.json", "w")  #formato?

for key in dictionary2:
    file.write(str(key) + ": " + str(dictionary2[key]) + "\n")
    
file.close()

NOW:  fill the query vector components with 1 if the word corresponding to the component is present in the query, or 0 if it's not present. We will also fill the documents vectors components with the TFIDF of each term. Then we will compute the similarity between the query vector and each one of the document vectors as the cosine:

 cos(theta) = (q*d)/(|q|*|d|)




In [ ]:
vocabulary_file = open("vocabulary.txt", "r", 
                       encoding = "utf-8")          
vocabulary = vocabulary_file.read()                 
vocabulary = vocabulary.split("\n")

# Create a vector for the query

v_query = [0]*(len('quante parole ci sono nel dizionario'+1)      # Initialize the components to 0


# To store the vectors for the documents
v_docs = list()

for doc_index in range(19124):       # Initialize the components to 0
    v_docs.append([])
    for j in range('quante parole ci sono nel dizionario'+1):
        v_docs[doc_index].append(0)


# Fill the query vector with 1 (if a word is present) or leave it to 0 (if a word is not present)        
for word in query:
    if word in vocabulary:
        term_id = vocabulary.index(word)
        v_query[int(term_id)] = 1

# Fill the documents vectors with the TFIDFs for the words        
for term_id in dictionary2:
    for tpl in dictionary2[term_id]:
        doc_index = int(tpl[0])
        tfidf = float(tpl[1])
        v_docs[doc_index][int(term_id)] = tfidf

# Close vocabulary file
vocabulary_file.close()

Dalla regia mi dicono (Benedetta): store the cosines  in a heap structure.
Computationally more efficient than it would be by using a simple list.
Ok se posso usare le librerie dovrebbe essere facile, atrimenti no idea.
Copiato spudoratamente dal web.

In [ ]:
import heapq
a_query = np.array(v_query)

# Create a heap of scores and a dictionary of scores
heap = list()
heapq.heapify(heap)
scores_dictionary = dict()

# For every document
for doc_index in range(19184):
    
    
    a_doc = np.array(v_docs[doc_index])
    
    # Compute the cosine of the angle between the query vector and the document vector
    cos = np.dot(a_query, a_doc)/(np.linalg.norm(a_query)*np.linalg.norm(a_doc))
    
    # Put the result in the dictionary
    scores_dictionary[doc_index] = cos
    
    # Update the heap
    heapq.heappush(heap, cos)

Top_k!! funzionerà? sono stanca per scrivere in Inglese

In [ ]:
#
k = 10

# Get the ordered list of top_k scores from the heap
top_k = heapq.nlargest(k, heap)

# Initialize list of top documents 
top_k_docs = list()

# Fill list of top documents
for i in range(len(top_k)):
    doc_index = list(scores_dictionary.keys())[list(scores_dictionary.values()).index(top_k[i])]
    top_k_docs.append(doc_index)
    del scores_dictionary[doc_index]

Now for the result, I can't figure out how to extract them without a dataframe

In [ ]:
#anche su internet h cercato ma mi viene solo così.
df_results = df.iloc[top_k_docs][["title", "description", "city", "url"]]
df_results.index = list(range(1, k+1))

df_results["similarity"] = [x for x in top_k]
